In [1]:
import os
import absl.logging

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from typing import Callable, Tuple
from tensorflow import keras
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

from functions.ciou import ciou_loss, ciou_metric
from functions.loading_data import parse_csv
from functions.loading_data import SMALLER_HEIGHT, SMALLER_WIDTH

absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
get_names = lambda root_path: [
    file_name.split('.')[0]
    for dir_path, _, file_names in os.walk(root_path)
    for file_name in file_names
]
get_paths = lambda path: [f'{os.path.join(root, file)}' for root, dirs, files in os.walk(path) for file in files]
base_dir = os.path.join('..', 'data', 'images_original_inception_resnet_v2_200x150_splitted_with_augmentation_10p')
train_dir = os.path.join(base_dir, 'training')
valid_dir = os.path.join(base_dir, 'validation')

In [3]:
def process_path(
        image_path: str,
        coords: np.ndarray) -> Tuple[tf.Tensor, np.ndarray]:
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    return img, coords


def load_and_preprocess_data(csv_file_path: str, img_dir: str) -> tf.data.Dataset:
    image_filenames, coordinates = parse_csv(csv_file_path)
    image_filenames = [os.path.join(img_dir, fname) for fname in image_filenames]

    return tf.data.Dataset \
        .from_tensor_slices((
            image_filenames,
            coordinates)) \
        .map(process_path) \
        .batch(64) \
        .shuffle(1024)

In [4]:
train_ds = load_and_preprocess_data(os.path.join('..', 'data', 'training_boxes.csv'), train_dir)
valid_ds = load_and_preprocess_data(os.path.join('..', 'data', 'validation_boxes.csv'), valid_dir)

In [5]:
def run_multi_attentive_model(
        model_factory: Callable,
        model_name: str,
        loss: Callable,
        metric: Callable,
        reduction_patience=5,
        monitor='val_ciou_metric'):
    MIN_DELTA = .001
    early_stopping = keras.callbacks.EarlyStopping(
        monitor=monitor,
        mode='max',
        patience=10,
        min_delta=MIN_DELTA)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor=monitor,
        mode='max',
        factor=0.95,
        min_delta=MIN_DELTA,
        patience=reduction_patience,
        min_lr=0.0005,
        verbose=1)
    model_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath='models/' + model_name + '_{epoch}',
        save_best_only=True)
    tensor_board = keras.callbacks.TensorBoard(log_dir=f'tensor_logs/{model_name}')
    model = model_factory(loss, metric)

    return model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=500,
        callbacks=[reduce_lr, model_checkpoint, tensor_board, early_stopping])

In [6]:
def get_baseline_model(loss: Callable, metric: Callable) -> keras.Model:
    def get_attention_module(prev: keras.layers.Layer) -> keras.layers.Layer:
        gap_layer = keras.layers.GlobalAveragePooling2D()(prev)
        gap_layer_res = keras.layers.Reshape((1, 1, 1536))(gap_layer)
        dense = keras.layers.Dense(1536, activation='relu')(gap_layer_res)
        dense = keras.layers.Dense(1536, activation='softmax')(dense)
        mul_layer = keras.layers.Multiply()([prev, dense])

        return mul_layer

    base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=(SMALLER_HEIGHT, SMALLER_WIDTH, 3))

    for layer in base_model.layers:
        layer.trainable = False

    attention_module = get_attention_module(base_model.output)
    flat = keras.layers.Flatten()(attention_module)
    locator_module = keras.layers.Dense(4608, activation='relu')(flat)
    locator_module = keras.layers.Dense(1024, activation='relu')(locator_module)
    locator_module = keras.layers.Dense(4, activation='sigmoid')(locator_module)

    model = keras.Model(base_model.input, outputs=locator_module)

    model.compile(optimizer='adam', loss=loss, metrics=[metric])

    return model

In [ ]:
history1 = run_multi_attentive_model(get_baseline_model, f'roi_detection_inception_1', ciou_loss, ciou_metric, 20)

Epoch 1/500
1831/1831 [==============================] - ETA: 0s - loss: 0.4995 - ciou_metric: 0.5005INFO:tensorflow:Assets written to: models\roi_detection_inception_1_1\assets


INFO:tensorflow:Assets written to: models\roi_detection_inception_1_1\assets


1831/1831 [==============================] - 394s 185ms/step - loss: 0.4995 - ciou_metric: 0.5005 - val_loss: 0.4624 - val_ciou_metric: 0.5377 - lr: 0.0010
Epoch 2/500
1452/1831 [======================>.......] - ETA: 51s - loss: 0.4727 - ciou_metric: 0.5273